In [1]:
import pandas as pd
import os
import sys 
!pip install BeautifulSoup4
import re
import nltk
nltk.download("stopwords")  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("french"))
from past.builtins import xrange
import numpy as np
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
%matplotlib inline
from sklearn import datasets
from sklearn.mixture import GaussianMixture

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train = pd.read_csv('./dataset/manitra_mails.csv',encoding='utf-8')

In [4]:
#train

In [5]:
def body_to_words(raw_body):
    # Convert to lower case, split into individual words
    words = raw_body.lower().split()                             
    #
    # In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("french"))     
    # 
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join(meaningful_words))

In [6]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["body"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(0,num_reviews):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append(body_to_words(str(train["body"][i])))

In [7]:
print("Cleaning and parsing the training set mail body...\n")
clean_train_reviews = []
for i in range(0,num_reviews):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0):
        print("body %d of %d\n" % ( i+1,num_reviews))                                                                   
    clean_train_reviews.append(body_to_words(str(train["body"][i])))

Cleaning and parsing the training set mail body...

body 1000 of 2664

body 2000 of 2664



In [8]:
#for body in clean_train_reviews:
#    print(body[0])

# Creating Features from a Bag of Words (Using scikit-learn)

In [9]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word", \
                             tokenizer = None, \
                             preprocessor = None, \
                             stop_words = None, \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [10]:
print(train_data_features.shape)

(2664, 5000)


In [11]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
#print(vocab)

In [12]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

#For each, print the vocabulary word and the number of times it 
#appears in the training set
#for tag, count in zip(vocab, dist):
#    print(count, tag)

In [18]:
def word_tokenizer(text):
        #tokenizes and stems the text
        tokens = word_tokenize(text)
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('french')]
        return tokens

In [19]:
def cluster_sentences(sentences, nb_of_clusters=5):
            tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                            stop_words=stopwords.words('french'),
                                            max_df=0.9,
                                            min_df=0.1,
                                            lowercase=False)
            #builds a tf-idf matrix for the sentences
            tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
            kmeans = KMeans(n_clusters=nb_of_clusters)
            kmeans.fit(tfidf_matrix)
            clusters = collections.defaultdict(list)
            for i, label in enumerate(kmeans.labels_):
                    clusters[label].append(i)
            return dict(clusters)

In [20]:
# MAIN
nclusters= 5
clusters = cluster_sentences(clean_train_reviews, nclusters)

In [21]:
#order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
#terms = tfidf_vectorizer.get_feature_names()

In [ ]:
#Gaussian Mixture Model
gmm = GaussianMixture(n_components=5)
gmm.fit(train_data_features)
proba_lists = gmm.predict_proba(train_data_features)

In [ ]:
gmm

In [ ]:
#Plotting
colored_arrays = np.matrix(proba_lists)
colored_tuples = [tuple(i.tolist()[0]) for i in colored_arrays]
fig = plt.figure(1, figsize=(7,7))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(train_data_features[:, 3], train_data_features[:, 0], train_data_features[:, 2],
          c=colored_tuples, edgecolor="k", s=50)
ax.set_xlabel("Petal width")
ax.set_ylabel("Sepal length")
ax.set_zlabel("Petal length")
plt.title("Gaussian Mixture Model", fontsize=14)

In [ ]:
#KMeans
km = KMeans(n_clusters=5)
km.fit(train_data_features)
km.predict(train_data_features)
labels = km.labels_
#Plotting
fig = plt.figure(1, figsize=(7,7))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(train_data_features[:, 3], train_data_features[:, 0], train_data_features[:, 2],
          c=labels.astype(np.float), edgecolor="k", s=50)
ax.set_xlabel("Petal width")
ax.set_ylabel("Sepal length")
ax.set_zlabel("Petal length")
plt.title("K Means", fontsize=14)